#### ASCII parsing


A .tr file is usually a space-separated log of packet events like:
+ 0.1 0 1 tcp 1000 ...
- 0.2 0 1 tcp 1000 ...


In [ ]:
import pandas as pd

cols = ['event', 'time', 'fromNode', 'toNode', 'protocol', 'size']
#change path name
ascii_file_path="/home/nourhen_dev/repos/Network_digital_twin/3N_10P_UDP/Output/three_nodes_NDT.tr"
df_ascii = pd.read_csv(ascii_file_path, delim_whitespace=True, names=cols, usecols=range(6))

# Filter send/receive
sent = df_ascii[df_ascii['event'] == '+']
recv = df_ascii[df_ascii['event'] == '-']

# Merge to compute delay per packet
merged = pd.merge(sent, recv, on=['fromNode', 'toNode', 'size'], suffixes=('_send', '_recv'))
merged['delay'] = merged['time_recv'] - merged['time_send']


/tmp/ipykernel_95914/1656836395.py:5: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df_ascii = pd.read_csv(ascii_file_path, delim_whitespace=True, names=cols, usecols=range(6))


In [31]:
df_ascii.head()

,event,time,fromNode,toNode,protocol,size
0,+,2.00000,/NodeList/0/DeviceList/0/$ns3::PointToPointNet...,ns3::PppHeader,(Point-to-Point,Protocol:
1,-,2.00000,/NodeList/0/DeviceList/0/$ns3::PointToPointNet...,ns3::PppHeader,(Point-to-Point,Protocol:
2,r,2.00208,/NodeList/1/DeviceList/0/$ns3::PointToPointNet...,ns3::PppHeader,(Point-to-Point,Protocol:
3,+,2.00208,/NodeList/1/DeviceList/0/$ns3::PointToPointNet...,ns3::PppHeader,(Point-to-Point,Protocol:
4,-,2.00208,/NodeList/1/DeviceList/0/$ns3::PointToPointNet...,ns3::PppHeader,(Point-to-Point,Protocol:


#### PCAP parsing

In [ ]:
from scapy.all import rdpcap
pcap_path="/home/nourhen_dev/repos/Network_digital_twin/3N_10P_UDP/Output/three_nodes-0-0.pcap"
packets = rdpcap("three_nodes-0-0.pcap")
for pkt in packets:
    print(pkt.time, pkt.summary())


2.000000 PPP / IP / UDP 10.1.1.1:49153 > 10.1.1.2:discard / Raw
2.004168 PPP / IP / UDP 10.1.1.2:discard > 10.1.1.1:49153 / Raw
2.500000 PPP / IP / UDP 10.1.1.1:49153 > 10.1.1.2:discard / Raw
2.504168 PPP / IP / UDP 10.1.1.2:discard > 10.1.1.1:49153 / Raw
3.000000 PPP / IP / UDP 10.1.1.1:49153 > 10.1.1.2:discard / Raw
3.004168 PPP / IP / UDP 10.1.1.2:discard > 10.1.1.1:49153 / Raw
3.500000 PPP / IP / UDP 10.1.1.1:49153 > 10.1.1.2:discard / Raw
3.504168 PPP / IP / UDP 10.1.1.2:discard > 10.1.1.1:49153 / Raw
4.000000 PPP / IP / UDP 10.1.1.1:49153 > 10.1.1.2:discard / Raw
4.004168 PPP / IP / UDP 10.1.1.2:discard > 10.1.1.1:49153 / Raw
4.500000 PPP / IP / UDP 10.1.1.1:49153 > 10.1.1.2:discard / Raw
4.504168 PPP / IP / UDP 10.1.1.2:discard > 10.1.1.1:49153 / Raw
5.000000 PPP / IP / UDP 10.1.1.1:49153 > 10.1.1.2:discard / Raw
5.004168 PPP / IP / UDP 10.1.1.2:discard > 10.1.1.1:49153 / Raw
5.500000 PPP / IP / UDP 10.1.1.1:49153 > 10.1.1.2:discard / Raw
5.504168 PPP / IP / UDP 10.1.1.2:discard

In [ ]:
import pyshark

cap = pyshark.FileCapture(pcap_path)
for pkt in cap:
    print(pkt.sniff_time, pkt.length, pkt.ip.src, pkt.ip.dst)

#### Flow monitor

In [ ]:
import xml.etree.ElementTree as ET
flowmon_path="/home/nourhen_dev/repos/Network_digital_twin/3N_10P_UDP/Output/flowmon_three_nodes.xml"


tree = ET.parse(flowmon_path)
root = tree.getroot()

# Iterate over flow statistics
for flow in root.findall(".//FlowStats/Flow"):
    flow_id = flow.get("flowId")
    tx_packets = int(flow.get("txPackets"))
    rx_packets = int(flow.get("rxPackets"))
    tx_bytes = int(flow.get("txBytes"))
    rx_bytes = int(flow.get("rxBytes"))
    lost_packets = int(flow.get("lostPackets"))

    # Convert delay and jitter from nanoseconds to seconds
    delay_sum_ns = float(flow.get("delaySum").replace('+', '').replace('ns', ''))
    delay_avg = delay_sum_ns / rx_packets / 1e9 if rx_packets > 0 else 0

    print(f"Flow ID: {flow_id}")
    print(f"  Tx Packets: {tx_packets}")
    print(f"  Rx Packets: {rx_packets}")
    print(f"  Lost Packets: {lost_packets}")
    print(f"  Tx Bytes: {tx_bytes}")
    print(f"  Rx Bytes: {rx_bytes}")
    print(f"  Avg Delay: {delay_avg:.6f} s")
    print("  ----------------------")


Flow ID: 1
  Tx Packets: 10
  Rx Packets: 10
  Lost Packets: 0
  Tx Bytes: 10520
  Rx Bytes: 10520
  Avg Delay: 0.002084 s
  ----------------------
Flow ID: 2
  Tx Packets: 10
  Rx Packets: 10
  Lost Packets: 0
  Tx Bytes: 10520
  Rx Bytes: 10520
  Avg Delay: 0.002084 s
  ----------------------


In [17]:
import xml.etree.ElementTree as ET
import pandas as pd

# Load the XML file
tree = ET.parse(flowmon_path)  # Replace with your file path
root = tree.getroot()

# Prepare a list to collect parsed flow data
flow_data = []

# Iterate over all <Flow> elements inside <FlowStats>
for flow in root.find('FlowStats').findall('Flow'):
    flow_id = int(flow.attrib['flowId'])
    time_first_tx = float(flow.attrib['timeFirstTxPacket'].replace('ns', '').replace('+', ''))
    time_first_rx = float(flow.attrib['timeFirstRxPacket'].replace('ns', '').replace('+', ''))
    time_last_tx = float(flow.attrib['timeLastTxPacket'].replace('ns', '').replace('+', ''))
    time_last_rx = float(flow.attrib['timeLastRxPacket'].replace('ns', '').replace('+', ''))
    delay_sum = float(flow.attrib['delaySum'].replace('ns', '').replace('+', ''))
    jitter_sum = float(flow.attrib['jitterSum'].replace('ns', '').replace('+', ''))
    last_delay = float(flow.attrib['lastDelay'].replace('ns', '').replace('+', ''))
    tx_bytes = int(flow.attrib['txBytes'])
    rx_bytes = int(flow.attrib['rxBytes'])
    tx_packets = int(flow.attrib['txPackets'])
    rx_packets = int(flow.attrib['rxPackets'])
    lost_packets = int(flow.attrib['lostPackets'])
    times_forwarded = int(flow.attrib['timesForwarded'])

    flow_data.append({
        'flow_id': flow_id,
        'time_first_tx_ns': time_first_tx,
        'time_first_rx_ns': time_first_rx,
        'time_last_tx_ns': time_last_tx,
        'time_last_rx_ns': time_last_rx,
        'delay_sum_ns': delay_sum,
        'jitter_sum_ns': jitter_sum,
        'last_delay_ns': last_delay,
        'tx_bytes': tx_bytes,
        'rx_bytes': rx_bytes,
        'tx_packets': tx_packets,
        'rx_packets': rx_packets,
        'lost_packets': lost_packets,
        'times_forwarded': times_forwarded
    })

# Optional: print or write to CSV

#df = pd.DataFrame(flow_data)
#df
# df.to_csv('flow_stats.csv', index=False)


In [18]:
classifier_info = {}
for flow in root.find('Ipv4FlowClassifier').findall('Flow'):
    flow_id = int(flow.attrib['flowId'])
    classifier_info[flow_id] = {
        'src': flow.attrib['sourceAddress'],
        'dst': flow.attrib['destinationAddress'],
        'src_port': flow.attrib['sourcePort'],
        'dst_port': flow.attrib['destinationPort'],
        'protocol': flow.attrib['protocol']
    }

# Merge with previous stats if needed
for row in flow_data:
    fid = row['flow_id']
    if fid in classifier_info:
        row.update(classifier_info[fid])


In [29]:
df_flowmon = pd.DataFrame(flow_data)
df_flowmon

,flow_id,time_first_tx_ns,time_first_rx_ns,time_last_tx_ns,time_last_rx_ns,delay_sum_ns,jitter_sum_ns,last_delay_ns,tx_bytes,rx_bytes,tx_packets,rx_packets,lost_packets,times_forwarded,src,dst,src_port,dst_port,protocol
0,1,2.000000e+09,2.002080e+09,6.500000e+09,6.502080e+09,20843200.0,0.0,2084320.0,10520,10520,10,10,0,0,10.1.1.1,10.1.1.2,49153,9,17
1,2,2.002080e+09,2.004170e+09,6.502080e+09,6.504170e+09,20843200.0,0.0,2084320.0,10520,10520,10,10,0,0,10.1.1.2,10.1.1.1,9,49153,17


In the context of IP networking, protocol number 17 corresponds to UDP (User Datagram Protocol)

In NS-3, a FlowProbe is used to gather low-level metrics at specific nodes or interfaces—like intermediate delays, hops, or internal observations—whereas FlowStats give end-to-end metrics (from sender to receiver).

| Field          | FlowStats                               | FlowProbes                                       |
| -------------- | --------------------------------------- | ------------------------------------------------ |
| Scope          | End-to-end (sender to receiver)         | Per-node or per-hop probe                        |
| Example Metric | `delaySum`, `txBytes`, `rxBytes`        | `delayFromFirstProbeSum`, `packets`, `bytes`     |
| Usefulness     | Essential for delay, jitter, throughput | Helpful for per-hop delay, loss, congestion path |

https://www.nsnam.org/docs/release/3.15/doxygen/classns3_1_1_flow_probe.html

The FlowProbe class is responsible for listening for packet events in a specific point of the simulated space, report those events to the global FlowMonitor, and collect its own flow statistics regarding only the packets that pass through that probe.

In [30]:
import xml.etree.ElementTree as ET

tree = ET.parse(flowmon_path)
root = tree.getroot()

probe_data = []
for probe in root.findall("FlowProbes/FlowProbe"):
    probe_index = probe.attrib["index"]
    for stat in probe.findall("FlowStats"):
        flow_id = stat.attrib["flowId"]
        packets = int(stat.attrib.get("packets", 0))
        bytes_ = int(stat.attrib.get("bytes", 0))
        delay = float(stat.attrib.get("delayFromFirstProbeSum", "+0ns").replace("+", "").replace("ns", ""))
        probe_data.append({
            "probe_index": int(probe_index),
            "flow_id": int(flow_id),
            "packets": packets,
            "bytes": bytes_,
            "delay_ns": delay
        })

# Now `probe_data` is a list of dictionaries you can turn into a DataFrame
import pandas as pd
df_probes = pd.DataFrame(probe_data)
df_probes


,probe_index,flow_id,packets,bytes,delay_ns
0,0,1,10,10520,0.0
1,0,2,10,10520,20843200.0
2,2,1,10,10520,20843200.0
3,2,2,10,10520,0.0
